In [12]:
from sklearn.neighbors import KDTree
import numpy as np
import random
import dask.array as da
import napari
import pandas as pd
import zarr

%gui qt

coord_path = 'C:/Users/cameron.trader/OneDrive - Allen Institute/Documents/Unet/0539049885-0072_r.csv'

coord = np.loadtxt(coord_path, skiprows=1, delimiter=',')

In [13]:
# z = zarr.open('C:/Users/camer/OneDrive - Allen Institute/Documents/Unet/0539050069-0035.zarr')
# for i in range(z.shape[0] // 260):
#     z[(i*260)-1:(i*260)+1, :, 0] = 5000
# for i in range (z.shape[1] // 260):
#     z[:, (i*260)-1:(i*260)+1, 0] = 5000
# image = da.from_zarr(z)

In [14]:
zarr_img_path = 'C:/Users/cameron.trader/OneDrive - Allen Institute/Documents/Unet/0539049885-0072.zarr'
image = da.from_zarr(zarr_img_path)

In [15]:
coord_df = pd.DataFrame(coord)
coord_df.columns = ['y', 'x', 'size']
size_over_50 = coord_df['size'] >= 50
coord_df = coord_df[size_over_50]
# distance_from_grid = 25
# buffer = 44
# before_vert = (coord_df['x'] - buffer) % 260 <= distance_from_grid
# after_vert = (coord_df['x'] - buffer) % 260 >= (260 - distance_from_grid)
# before_horiz = (coord_df['y'] - buffer) % 260 <= distance_from_grid
# after_horiz = (coord_df['y'] - buffer) % 260 >= (260 - distance_from_grid)
# coord_df = coord_df[(before_horiz | after_horiz) | (before_vert | after_vert)]
coord_s50 = coord_df.to_numpy()
print(coord_s50.shape)

(28898, 3)


In [16]:
kdt = KDTree(coord_s50[:,:2], leaf_size=30, metric='euclidean')

close_points = []
close_set =[]
indices_to_drop = []

close = kdt.query_radius(coord_s50[:,:2], r=10)
for i in close:
    if i.size > 1:
        close_points.append(list(i))
for i in close_points:
    if not i in close_set:
        close_set.append(i)
print("Close points are: ", str(close_set))
for i in close_set:
    while len(i)>1:
        index = random.randrange(len(i))
        indices_to_drop.append(i[index])
        del i[index]

print(str(len(indices_to_drop)),"points to be removed:", str(indices_to_drop))
# print("These points are located at:")
# np.set_printoptions(suppress=True)
# for i in indices_to_drop:
#     print(coord[indices_to_drop])
# np.set_printoptions(suppress=False)
coord_s50_clean = np.array([i for j, i in enumerate(coord_s50) if j not in indices_to_drop])
coord_s50_removed = np.array([i for j, i in enumerate(coord_s50) if j in indices_to_drop])

print(str(coord_s50.shape[0]-coord_s50_clean.shape[0]), "points removed")

Close points are:  [[1454, 90], [1452, 91], [178, 2362], [337, 2594], [719, 472], [565, 811], [2767, 569], [819, 2891], [2994, 1038], [1720, 1512], [1739, 1527], [1959, 1772], [2117, 3640], [2693, 2480], [2712, 2502], [2690, 2844], [3028, 3167], [3119, 3277], [5277, 3639], [3804, 4034], [4020, 5458], [5555, 4227], [4228, 5552], [4229, 5551], [4316, 4434], [4407, 6179], [4460, 4678], [4721, 4506], [4774, 6491], [4850, 6932], [4925, 4978], [4988, 7160], [5599, 5806], [5819, 6075], [6253, 6158], [6267, 6419], [6457, 6305], [8560, 6341], [7059, 6909], [7063, 6916], [8984, 6927], [6928, 8979], [6929, 8981], [6931, 8983], [9593, 7075], [9594, 7076], [7193, 7103], [7431, 7574], [7664, 7592], [7774, 8000], [8026, 7812], [8072, 7861], [8090, 7889], [8163, 10641], [10640, 8164], [8165, 10638], [8509, 8602], [11774, 8721], [8738, 9063], [8763, 9158], [8834, 9336], [9452, 8902], [11946, 8975], [12415, 9591], [9592, 12413], [9953, 13252], [13253, 9954], [11472, 11270], [14152, 11338], [14153, 11339

In [17]:
# buffer = 44

# vectors = np.zeros((image.shape[0]//260 + image.shape[1]//260, 2, 2))
# vectors[:115,0,0] =+ 1
# vectors[:115,:,0] = vectors[:115,:,0] * np.vstack(np.arange(image.shape[0]//260)) * 260 + buffer
# vectors[:115,1,0] = 0
# vectors[:115,1,1] = image.shape[0]

# vectors[115:,0,1] =+ 1
# vectors[115:,0,1] = vectors[115:,0,1] * np.arange(image.shape[1]//260) * 260 + buffer
# vectors[115:,1,0] = image.shape[0]

In [19]:
viewer = napari.view_image(image[:, :, 0], name='slice')
viewer.add_points(coord_s50_clean[:, :2], size=2, face_color='red')
# viewer.add_points(coord_s50_removed_15[:, :2], size=2, face_color='blue')
# viewer.add_points(coord_s50_removed_15[:, :2], size=200, face_color='blue')
viewer.add_points(coord_s50_removed[:, :2], size=2, face_color='yellow')
viewer.add_points(coord_s50_removed[:, :2], size=200, face_color='yellow')
# viewer.add_vectors(vectors, edge_width=3)

C:\Users\cameron.trader\Anaconda3\envs\napari\lib\site-packages\napari\_vispy\vispy_image_layer.py:198: UserWarning: data shape (30000, 40000) exceeds GL_MAX_TEXTURE_SIZE 32768 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


<Points layer 'Points [2]' at 0x1be4b181520>

In [45]:
np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_clean.csv', coord_clean, delimiter=",")

np.savetxt('/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/0539050069-0035_r_removed.csv', coord_removed, delimiter=",")

NameError: name 'coord_clean' is not defined